In [1]:
import pandas as pd
from IPython.display import HTML
def printmd(string):
    from IPython.display import Markdown, display
    display(Markdown(string))

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

### postgresql + python ###
import psycopg2 as pg2

class psql_py:
    def __init__(self):
        self.conn = self.connect_db()
        self.cur = self.conn.cursor()
        self.cur.execute("set search_path = mimiciii;") 
        
    def connect_db(self):
        try:
            conn = pg2.connect("host = localhost dbname=mimic user=postgres port=5432")
            #print('database connected')
            return conn
        except:
            print("I am unable to connect to the database")          
            
    def exe_sql (self, exe_sql_str):
        return pd.read_sql(exe_sql_str,self.conn)
    
    def db_query(self, exe_sql_str, mode='sql'):
        if mode == 'pd':
            return_pd = self.exe_sql (exe_sql_str)
            self.close_db()
            return return_pd
        else: 
            self.cur.execute(exe_sql_str)
            self.close_db()
            
    def close_db (self):
        self.cur.close()
        # commit the changes
        self.conn.commit()
        #print('database disconnected')


In [98]:
dict_variable_id = pd.read_csv('./csv/item_ids_dict.csv') # variables related to extubation 

#supervised by Young-Jae Cho at Seoul National University Bundang Hostpital 

In [112]:
db = psql_py() 
print('extration each variable from chartevent')
for i, row in dict_variable_id.iterrows(): #from chartevent
    var_name = row['name']
    var_name = var_name.replace('\b/','_')
    var_id = row['id']
    str_sql = ('select chartevents.icustay_id, chartevents.itemid, chartevents.charttime, chartevents.value '+
               'from chartevents, ptsinfo_vent '+ 
               'where chartevents.icustay_id = ptsinfo_vent.icustay_id and itemid in ('+str(var_id)+
              ') and ptsinfo_vent.admittime <= chartevents.charttime and ptsinfo_vent.dischtime >= chartevents.charttime')
    var_record = db.db_query(str_sql,'pd')
    var_record.to_csv('./variable/chartevent_var_'+var_name+'.csv')
    print(var_name,var_id)
    
print('extration each variable from labevent')    
for i, row in dict_variable_id.iterrows(): #from labevent
    var_name = row['name']
    var_name = var_name.replace('\b/','_')
    var_id = row['id']
    str_sql = ('select labevents.subject_id, labevents.itemid,labevents.charttime, labevents.value from labevents, ptsinfo_vent
                ' where labevents.subject_id = ptsinfo_vent.subject_id and itemid in ('+str(var_id)+
              ')and ptsinfo_vent.admittime <= labevents.charttime and ptsinfo_vent.dischtime >= labevents.charttime')
    var_record = db.db_query(str_sql,'pd')
    if len(var_record)>0:
        var_record.to_csv('./variable/chartevent_var_'+var_name+'.csv')
    print(var_name,var_id)
    
    

GCS total 198, 226755, 227013
GCS-eye opening 220739, 226756, 227011, 184
GCS - Verbal Response 223900, 228112, 226758, 227014, 723
GCS - Motor Response 223901, 226757, 227012, 454
RASS 228096
